### Installation

In [2]:
# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [3]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
models_name = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Llama-3.2-3B-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    # not pre quantized models
    "unsloth/Meta-Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.2-1B",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
] # More models at https://huggingface.co/unsloth



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models_name[6],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = userdata.get('HF_LLAMA') # you need to request access to Llama models and use here your huggingface key
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

### Direct Preference Optimization (DPO): Overview

**Direct Preference Optimization (DPO)** is a fine-tuning strategy for large language models (LLMs) that optimizes models to align with human preferences — *without needing reinforcement learning or reward models*, unlike traditional methods like RLHF (Reinforcement Learning from Human Feedback).

#### Main Concepts

* **Goal**: Learn to prefer outputs that humans rate more favorably.
* **Data**: Pairs of model responses (preferred vs. dispreferred), based on human or synthetic feedback.
* **No reward model needed**: Unlike RLHF, DPO uses these preferences directly for supervised training.

#### How it Works

1. **Input**: Triplets of (prompt, preferred response, dispreferred response).
2. **Loss function**: The model is trained to assign higher probabilities to preferred responses.
3. **Efficiency**: DPO simplifies training by avoiding unstable reward model learning or complex policy optimization.

#### Advantages

* **Stable and simple**: No reinforcement learning tricks required.
* **Efficient**: Uses standard supervised learning infrastructure.
* **Scalable**

#### Reference

Rafailov, R., Sharma, A., Mitchell, E., Manning, C. D., Ermon, S., & Finn, C. (2023). Direct preference optimization: Your language model is secretly a reward model. *Advances in Neural Information Processing Systems, 36*, 53728-53741.

### Data preparation

We use the [Ultra Feedback dataset](https://huggingface.co/datasets/trl-lib/ultrafeedback_binarized).

Creating a dataset for DPO (Direct Preference Optimization) training involves collecting and structuring data in a specific format that allows a model to learn preferences between pairs of responses. DPO is an alternative to reinforcement learning with human feedback (RLHF), and it works by optimizing a reward model implicitly, without requiring explicit reward scores — it only needs preference comparisons.

Each entry contains:
* A prompt (or query/input),
* A chosen response (the one preferred by a human annotator or a heuristic),
* A rejected response (a plausible but less-preferred alternative).

The prompt field is optional. Sometimes, it is encoded into the responses. In some datasets derived from multi-turn conversations, the "prompt" is implicitly encoded in the previous dialogue turns.

In [7]:
from datasets import load_dataset

dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train[0:1000]") # We consider only a subset of 1,000 examples
dataset.to_csv("ultrafeedback_binarized.csv")


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

3933991

In [8]:
import pprint

pprint.pprint(dataset[0]['chosen'])
pprint.pprint(dataset[0]['rejected'])

[{'content': 'Use the pygame library to write a version of the classic game '
             'Snake, with a unique twist',
  'role': 'user'},
 {'content': "Sure, I'd be happy to help you write a version of the classic "
             "game Snake using the pygame library! Here's a basic outline of "
             'how we can approach this:\n'
             '\n'
             "1. First, we'll need to set up the game display and create a "
             "game object that we can use to handle the game's state.\n"
             "2. Next, we'll create the game's grid, which will be used to "
             "represent the game board. We'll need to define the size of the "
             'grid and the spaces within it.\n'
             "3. After that, we'll create the snake object, which will be used "
             "to represent the player's movement. We'll need to define the "
             'size of the snake and the speed at which it moves.\n'
             "4. We'll also need to create a food object, whic

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Train"></a>
### Train DPO the model
Now let's use Huggingface TRL's `DPOTrainer`! More docs here: [TRL DPO docs](https://huggingface.co/docs/trl/dpo_trainer). We do 200 steps to speed things up.

In [10]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        # num_train_epochs = 3,
        max_steps = 100,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 25,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    beta = 0.1,
    train_dataset = dataset,
    # eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model.

In [ ]:
model.save_pretrained("lora_dpo_model")  # Local saving
tokenizer.save_pretrained("lora_dpo_model")

# for saving the merged model (model + adapter)
#model.save_pretrained_merged("merged_lora_dpo_model", tokenizer, save_method = "merged_16bit",)
#model.save_pretrained_merged("merged_lora_dpo_model", tokenizer, save_method = "merged_4bit",)

('lora_dpo_model/tokenizer_config.json',
 'lora_dpo_model/special_tokens_map.json',
 'lora_dpo_model/tokenizer.json')